<a href="https://colab.research.google.com/github/tluxxx/PortfolioExperiments/blob/main/GebertIndicatorTest(1993_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gebert Indicator -  Backtest Results

#1. General Preparations

In [233]:
!pip install ecbdata

In [234]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [235]:
# import data processing libraries
import pandas as pd
import numpy as np
import datetime as dt

# import plotting libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import data providing libraries
import yfinance as yf
import pandas_datareader as pdr
from ecbdata import ecbdata

# import supporting modules
from scipy.stats import pearsonr

#2. Downloading required data and data preparations

## 2.1 DAX-data & trading/analyses days

In [236]:
# importing the DAX-data
start_period, end_period = '1992-01-01', '2024-07-31'   # download data for analyses
start_assessment = '1993-01-01'                         # start of Indicator assessment
start_analyses = '1991-12-31'

ticker ='^GDAXI'
dax_d = yf.download(ticker, start=start_period, end=end_period)

# adding a field data to support resampling
dax_d['date'] = pd.to_datetime(dax_d.index).date

[*********************100%%**********************]  1 of 1 completed


In [237]:
# General preparations

# calculation of first and last trading day of a month (last trading day = date of data-observation and analyses), tobulation of result
dates_start_m = pd.date_range(start_period, end_period, freq ='MS')
dates_end_m = pd.date_range(start_period, end_period, freq ='M')
trade_dates_m = pd.to_datetime(dax_d.resample('MS').first()['date'].to_list())

# generation of dataframe for results (params)
params = pd.DataFrame({'trade_dates':trade_dates_m,
                       'end_dates': dates_end_m},
                      index=dates_start_m)

# start generating the reselts dataframe (params)
params['analyses_dates'] = params.end_dates.shift(1)
params.loc[params.index[0],'analyses_dates'] = dt.datetime.strptime(start_analyses, '%Y-%m-%d')
params.drop('end_dates', axis=1, inplace=True)
analyses_dates_m = params['analyses_dates'].tolist()
final_analyses_date = analyses_dates_m[-1]


## 2.2 Inflation Data

In [238]:
# reading and preparing inflation data for 1991-2024

# preparations
start_ecb_data ='1997-01-01'
switch_date_inf ='1998-01-31'

# data from German Statistical Office (1992-2024) [note: only CPI data available prior to 1992]
data_ger = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/GermanInflationData_(1992-2024)_CPI_12m.xlsx', sheet_name='DATA')
inf_GER = pd.DataFrame({'analyses_dates': data_ger['end_date'].to_list(),
                        'inflation_12m': data_ger['inflation_12m'].to_list()},
                       index = data_ger['start_date'].to_list())

# download data from ECB (1997-2024)
data_key = 'ICP.M.U2.N.000000.4.ANR'  # HICP - Overall index, Euro area (changing composition), Monthly
inf_eu = ecbdata.get_series(data_key, start=start_period, end=end_period)
inf_eu.TIME_PERIOD = pd.to_datetime(inf_eu.TIME_PERIOD)
inf_eu = inf_eu.set_index('TIME_PERIOD')
inf_eu = inf_eu.OBS_VALUE.astype(float)
end_dates = pd.date_range(start_ecb_data, end_period, freq ='M')
inf_EU = pd.DataFrame({'analyses_dates': end_dates.tolist(),
                       'inflation_12m': inf_eu},
                      index=inf_eu.index)

# Adaptation of the GER-data-frame to start_period
inf_GER = inf_GER.loc[start_period:]
inf_GER.inflation_12m.astype(float)

# Combination of GER and ECB time-series at a cut-off-date
df1 = inf_GER.loc[start_period:switch_date_inf]
df2 = inf_EU.loc[switch_date_inf:]
inf_EUa = pd.concat([df1,df2])


## 2.3. Interest Rates

In [239]:
# reading and preparing interest data from Central Bank Sources

# preparation
switch_date_int = '1998-12-31'                           # switch from German Bundesbank-data to ECB-data
start_upload = dt.datetime.strptime(start_period, '%Y-%m-%d') - dt.timedelta(31)   # ensuring data are available for early 1993
dates_d = pd.date_range(start_upload, switch_date_int)   # data-window
init_d = np.nan

# upload German Bundesbank discount rates (interest-value per last day of the month) (from previously downloaded file)
data_db = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/DiscountRatesGermanBundesbank (until 1998).xlsx',sheet_name='DATA' )
data_db['date'] = pd.to_datetime(data_db['dates'])

# transfer of relevant monthly data (starting from start_upload) to a data-frame
disc_temp = data_db[data_db['date'] >= start_upload]
disc_r = pd.DataFrame({'date':disc_temp.date.tolist(),
                       'interest':disc_temp.interest.tolist()})

# conversion to daily data (approximation: if interest rate changes during a month, the change will be implemented only at last day of this month)
int_temp1 = pd.DataFrame({'date': dates_d,
                          'interest': init_d})
int_temp2 = pd.merge(int_temp1, disc_r, how='left', on='date')['interest_y']    # two columns interest, after merge only interest_y from disc_r is relevant
int_temp2.ffill(inplace=True)
db_int = pd.DataFrame({'interest':int_temp2.tolist()},
                      index = dates_d)
db_int = db_int.interest.astype(float)

# fetch ECB key interest data (for each calendar day)
# key interest rate: Main refinancing operations - Minimum bid rate/fixed rate (date of changes) - Level, Euro area (changing composition), Daily
data_key = 'FM.D.U2.EUR.4F.KR.MRR_RT.LEV'
ecb_int = ecbdata.get_series(data_key, start=start_period, end=end_period)
ecb_int.TIME_PERIOD = pd.to_datetime(ecb_int.TIME_PERIOD)
ecb_int = ecb_int.set_index('TIME_PERIOD')
ecb_int = ecb_int.OBS_VALUE.astype(float)

# aggregation of German Discount rate with ECB key interest rate
intr_agg = pd.concat([db_int, ecb_int])
interest_agg = intr_agg[intr_agg.index.isin(analyses_dates_m)]
interest_agg.name = 'interest'

## 2.3 Exchange rates EURO - USD

In [240]:
# Exchange rates EURO-USD

# preparation
switch_date_curr = '1999-01-01'                     # switch from German Bundesbank to ECB data
dates_d = pd.date_range(start_upload, end_period)   # data-window
init_d = np.nan

# uploading USD-DM historical exchange rates (previously downloaded from German Bundesbank)
data_db = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/ExchangeRates_(USD_DM)_(until 1998).xlsx', sheet_name='DATA')

# processing USD-DM data  (daily data for each German Bank Day)
data_db[data_db['data'] =='.'] = np.nan
data_db.rename(columns = {'data':'rate'}, inplace = True)
data_db.dropna(inplace=True)
data_db.set_index('date', inplace=True)
data_db = data_db.rate.astype(float)

# conversion of relevant USD-DM data (from start_period) to EUR-USD
usd_dm_db = data_db.loc[start_upload:]            # USD-DM data
eur_usd_db = 1 / usd_dm_db.div(1.95583)           # reverting the currency pair and transforming to EURO
eur_usd_db.name = 'rate'

# importing the relevant EURO-USD historical exchange rates from FRED (daily data for each US bank day) (series starts at switch_date_curr)
eur_usd_fred = pdr.data.DataReader('DEXUSEU', 'fred', switch_date_curr, end_period)
eur_usd_fred.rename(columns = {'DEXUSEU':'rate'}, inplace = True)
eur_usd_fred['rate'].ffill(inplace=True)
eur_usd_fred = eur_usd_fred['rate']

# combining Bundesbank and FRED data to one timeseries and storing in an aggregated time-series
eur_usd = pd.concat([eur_usd_db, eur_usd_fred])
eur_usd_temp1 = pd.DataFrame({'rate':init_d}, index=dates_d)
eur_usd_temp2 = pd.merge(eur_usd_temp1, eur_usd, how='left', left_index=True, right_index=True)['rate_y']
eur_usd_temp2.name ='rate'
eur_usd_temp2.ffill(inplace=True)
eur_usd_agg = eur_usd_temp2[eur_usd_temp2.index.isin(analyses_dates_m)]
eur_usd_agg.name = 'exchange_rate'


# 3. Gebert-Indicator and Trade Positions

## 3.1 Calculation

In [241]:
# collecting raw data for Gebert/Indicator-Calculation

# collection of monthly dax-data   [close of the first trading day of the month]
params['dax'] = dax_d[dax_d.index.isin(trade_dates_m)]['Close'].tolist()

# collection of monthly inflation-data, shifting, to reflect that at first trading day of a month inflation date are not ready yet!)
params['infl_GER'] = inf_GER['inflation_12m'].shift(1)
params['infl_EUa'] = inf_EUa['inflation_12m'].shift(1)

# collection of monthly interest-data
params['int'] = interest_agg.tolist()

# collection of exchange rate data
params['eur_usd'] = eur_usd_agg.tolist()

# collection input for saisonality
params['season'] = params.index.month


In [242]:
# calculation of subindicators, the resulting Gebert-Indicator and Positions

# subindicator interest: +1: if last adaptation of CB's benchmark interest rate was a reduction, 0: otherwise
interest_chg = params['int'].diff()
params['i_int'] = np.select([interest_chg < 0, interest_chg > 0, interest_chg == 0], [1, 0, np.nan])
params['i_int'].ffill(inplace=True)

# subindicator inflation: +1: if inflation rate declined, compared to -12m value, 0: otherwise
params['i_infl_GER'] = np.where(params['infl_GER'] < params['infl_GER'].shift(12), 1, 0)
params['i_infl_EUa'] = np.where(params['infl_EUa'] < params['infl_EUa'].shift(12), 1, 0)

# subindicator exchange rates USD/EUR: +1: if USD got stronger compared to -12m, 0: otherwise
params['i_eur_usd'] = np.where(params['eur_usd'] < params['eur_usd'].shift(12), 1, 0)

# subindicator seasonality: +1: during [November -April], 0: otherwise
params['i_season'] = np.where(params['season'].isin([11,12,1,2,3,4]), 1, 0)

# Gebert-Indicator and Trade postions
i_GER = ['i_int', 'i_infl_GER', 'i_eur_usd', 'i_season']
i_EUa = ['i_int', 'i_infl_EUa', 'i_eur_usd', 'i_season']
params['GI_GER'] = params[i_GER].sum(axis=1)
params['GI_EUa'] = params[i_EUa].sum(axis=1)

# calculation of trade positons: GI=3,4 stay/go LONG; GI=0,1 stay/go FLAT, GI=2 keep positions
cond_GER = [params['GI_GER'] <= 1, params['GI_GER'] >= 3, params['GI_GER'] == 2]
cond_EUa = [params['GI_EUa'] <= 1, params['GI_EUa'] >= 3, params['GI_EUa'] == 2]
choices = [0, 1, np.nan]
params['pos_GER'] = np.select(cond_GER, choices)
params['pos_EUa'] = np.select(cond_EUa, choices)
params['pos_GER'].ffill(inplace=True)
params['pos_EUa'].ffill(inplace=True)

#finetuning:
col_list = ['i_int', 'GI_GER', 'GI_EUa', 'pos_GER', 'pos_EUa']
params[col_list] = params[col_list].astype(int)

# limitation to assessment-period
params = params.loc[start_assessment:].copy()


## 3.2. Visualisation of Gebert Indicator and of Sub-indicators

In [243]:
# Helper function for plotting indicator

def plot_int(pr, text_l, name, name_ind):
  ''' plotting (sub)-indicator value and color-coding of the proposed trade-position
  Args:
    pr (DataFrame):   dataframe with all required values (prices, sub-indicators, Geber-indicator, positions etc...)
    text_l:           long-text, describing the visualized (sub)-indicator
    name:             name of column in pr, that contains the indicator values
    name_ind:         name of column in pr, that contains the proposed trade postion (0 = FLAT, 1=LONG)
  Returns:
    (none)
  '''

  main_title= f'timeline of {text_l} ({name}) and sub-indicator value'
  sub_title = f'instrument: {ticker}, analyzed  from: {start_assessment} to: {end_period} '
  title = main_title + '<br><br><sup>' + sub_title + '</sup>'

  # preparation
  x = pr[name]
  ind = pr[name_ind]

  y1 = np.where(((pr[name_ind] == 1) | (pr[name_ind].pct_change() < 0)), pr[name], np.nan)
  y2 = np.where(((pr[name_ind] == 0) | (pr[name_ind].pct_change() > 1)), pr[name], np.nan)
  colors = ['green','red']

  # plotting
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=params.index, y=y1, line_width = 3, line_color =  colors[0], name = f'{name} - (sub)-indicator = 1 (supporting LONG)'))
  fig.add_trace(go.Scatter(x=params.index, y=y2, line_width = 3, line_color = colors[1], name = f'{name} - (sub)-indicator = 0 (supporting FLAT)'))

  fig.update_layout(template='plotly_dark', autosize=False, width=1600, height=400)
  fig.update_layout(title=title, xaxis_title='time', yaxis_title=f'subindcator {name_ind}', legend_title='Positions')
  fig.show()

In [244]:
# Visualiation sub-indicator inflation (Germany)
text_long, parameter, indicator = 'inflation (Germany)', 'infl_GER', 'i_infl_GER'
plot_int(params, text_long, parameter, indicator)

In [245]:
# Visualiation sub-indicator inflation (EU-aggregated)
text_long, parameter, indicator = 'inflation (EU-aggregated)', 'infl_EUa', 'i_infl_EUa'
plot_int(params, text_long, parameter, indicator)

In [246]:
# Visualiation sub-indicator central-bank-interest-rate
text_long, parameter, indicator = 'Central Bank interest rate', 'int', 'i_int'
plot_int(params, text_long, parameter, indicator)


In [247]:
# Visualiation sub-indicator EUR-USD-exchange-rate
text_long, parameter, indicator = 'EUR-USD-exchange-rate (aggregated)', 'eur_usd', 'i_eur_usd'
plot_int(params, text_long, parameter, indicator)

In [248]:
# Visualiation Gebert indicator (GER-data for inflation)
text_long, parameter, indicator = 'GEBERT-indicator (inflation data: Germmany)', 'GI_GER', 'pos_GER'
plot_int(params, text_long, parameter, indicator)

In [249]:
# Visualiation Gebert indicator (EU-aggregated-data for inflation)
text_long, parameter, indicator = 'GEBERT-indicator (inflation data: EU-aggregated)', 'GI_EUa', 'pos_EUa'
plot_int(params, text_long, parameter, indicator)

# 4. Results

In [250]:
# helper fuction to calculate PnL of Gebert indicator system (accumulated returns) and of B&H for a given start date

def pnl_calc(pr,start):
  ''' calculation of PnL of Gebert-indicator-strategy (accumulated returns) and PnL of Buy&H (w/o fees)
      without fees [using GER and EU-aggregated inflation data]
  Args:
    pr (DataFrame):   dataframe with all required values (prices, sub-indicators, Geber-indicator, positions etc...)
    start:            start date of the asssesment of the data
  Returns:
      pr_c:           dataframe supplemented by the PnL data of GI-strategy and B&H
  '''

  prc = pr.loc[start:].copy()
  # PnL-strategy
  prc['pnl_GER'] = (1 + prc['dax'].pct_change() * prc['pos_GER']).cumprod()
  prc['pnl_EUa'] = (1 + prc['dax'].pct_change() * prc['pos_EUa']).cumprod()
  # Buy and Hold data as reference
  prc['pnl_buh'] = (1 + prc['dax'].pct_change()).cumprod()
  return prc

def plot_pnl_ind(pr, mode, source, start, end, ticker):
  ''' plot of PnL of Gebert-indicator-strategy (accumulated returns)and plot of PnL of B&H (w/o fees)
      plot of Gebert-indicator-values and of positions
  Args:
    pr (DataFrame):   dataframe with all required values (prices, sub-indicators, Geber-indicator, positions etc...)
    mode:             if set to 'log' the PnL will be displayed in logarithmic scale
    ticker:           ticker of the instrument
    start:            start date of the asssesment of the data
  Returns:
      none:           plot of the PnL and subplots of Gebert-indicator and position
  '''
  # preparation
  name_pnl = 'pnl_' + source
  name_ind ='GI_' + source
  name_pos ='pos_' + source

  # plotting
  if source == 'GER':
    data_source = 'inflation-data from Germany'
  elif source == 'EUa':
    data_source = 'inflation-data from EU-aggregated'

  main_title = f'development of PnL position of Gebert-Indicator-Strategy ({data_source}) vs. B&H'
  sub_title = f'instrument: {ticker}, analyzed  from: {start} to:{end}'
  title = main_title + '<br><br><sup>' + sub_title + '</sup>'

  fig = make_subplots(rows=3, cols=1, row_heights=[0.7,0.1,0.1], shared_xaxes=True, vertical_spacing=0.02)
  fig.add_trace(go.Scatter(x=pr.index, y=pr[name_pnl], name = f'Gebert Strategy ({data_source}) - accumulated returns'), row=1, col=1)
  fig.add_trace(go.Scatter(x=pr.index, y=pr['pnl_buh'], name = 'Buy and Hold'), row=1, col=1)
  fig.add_trace(go.Scatter(x=pr.index, y=pr[name_ind], line_shape='hv', line_color='lightgreen', name = f'Gebert-Indicator-Value ({data_source})'), row=2, col=1)
  fig.add_trace(go.Scatter(x=pr.index, y=pr[name_pos], line_shape='hv', line_color='green', fill='tozeroy', name=f'position ({data_source}'), row=3, col=1)

  fig.update_layout(title=title, legend_title ='Sources', template='plotly_dark', autosize=False, width=1500, height=1000)

  if mode =='log':
    fig.update_yaxes(type='log', title_text='log(PnL)', row=1, col=1)
  else:
    fig.update_yaxes(title_text='PnL', row=1, col=1)

  fig.update_yaxes(title_text='GI', row=2, col=1)
  fig.update_yaxes(title_text='position', row=3, col=1)
  fig.update_xaxes(title_text='time', row=3, col=1)
  fig.show()


In [251]:
# PnL-calculation of overal period (start_asessment to end_period)
params_c = pnl_calc(params,start_assessment)

#visualisation
source = 'GER'
plot_pnl_ind(params_c, 'nonlog', source, start_assessment, end_period, ticker)

In [252]:
#visualisation
source = 'EUa'
plot_pnl_ind(params_c, 'nonlog', source, start_assessment, end_period, ticker)

In [253]:
# DEEPER-DIVE: Drawdown (%) (based on European inflation data)
x0 = ((params_c['pnl_EUa'] / params_c['pnl_EUa'].expanding().max()) - 1) * 100
x1 = ((params_c['pnl_buh'] / params_c['pnl_buh'].expanding().max()) - 1) * 100

main_title = 'drawdown Gebert Strategy (inflation data: EU-aggregated) vs. Buy & Hold'
sub_title = f'instrument: {ticker}   from   {start_assessment}   to   {end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=x0.index, y=x0, mode='none', fill='tozeroy', fillcolor='rgba(0, 255, 0, 0.3)', name='Gebert-Strategy'))
fig.add_trace(go.Scatter(x=x1.index, y=x1, mode='none', fill='tozeroy', fillcolor='rgba(255, 255, 0, 0.3)', name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark',autosize=False, width=1500, height=400)
fig.update_layout(title=title, xaxis_title='date', yaxis_title='drawdown (%)')
fig.show()

In [254]:
# DEEPER DIVE: visualisation with log scale for Pnl

source = 'EUa'
plot_pnl_ind(params_c, 'log', source, start_assessment, end_period, ticker)

In [255]:
# DEEPER-DIVE: Outperformance Gebert-Strategy vs B&H over time

# preparation
x = (params_c['pnl_EUa'] - params_c['pnl_buh']) / params_c['pnl_buh'] *100

main_title = f'development of outferformance (PnL_Strat - PnL_B&H) / PnL_B&H) over time, based on Gebert-Indicator-Strategy (inflation-data: EU-aggregated)'
sub_title = f'instrument: {ticker}, analyzed  from: {start_assessment} to:{end_period}'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

# plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=x.index, y=x, name = f'Gebert Strategy (inflation-data EU-aggregated) - accumulated returns'))
fig.update_layout(template = 'plotly_dark',autosize=False, width=1500, height=400)
fig.update_layout(title=title, xaxis_title='date', yaxis_title='Outperformance (%)')
fig.show()

In [256]:
# DEEPER DIVE: Study of alternative start days
start_date_alternative = '2015-01-01'
source = 'EUa'
params_c = pnl_calc(params,start_date_alternative)
plot_pnl_ind(params_c, 'nonlog', source, start_date_alternative, end_period, ticker)

In [257]:
# deeper dive simulation of the Outperformance of Gebert-indicator strategy vs B&H at various start dates for strategy implementation
# verification period
start_verification = '1993-01-01'
end_verification = '2019-12-31'

# preparations
test_starts = pd.date_range(start=start_verification, end=end_verification, freq='MS')
results_strat, results_buh = [], []

# calculations
for start in test_starts:
  pr_filt = params.loc[start:].copy()
  pr_test = pnl_calc(pr_filt, start)
  results_strat.append(pr_test['pnl_EUa'].iloc[-1])
  results_buh.append(pr_test['pnl_buh'].iloc[-1])

#compilation of the final results
results = pd.DataFrame({'pnl_dyn_EUa':results_strat,
                        'pnl_dyn_buh': results_buh},
                       index= test_starts)
x = (results['pnl_dyn_EUa'] - results['pnl_dyn_buh']) / results['pnl_dyn_buh'] *100

# plotting
main_title = f'development of outferformance (PnL_Strat - PnL_B&H) / PnL_B&H) depending of start of strategy impementation over time, based on Gebert-Indicator-Strategy (inflation-data: EU-aggregated)'
sub_title = f'instrument: {ticker}, analyzed  from: various start_dates to:{end_period}'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=x.index, y=x, name = f'Gebert Strategy (inflation-data EU-aggregated) - accumulated returns'))
fig.update_layout(template = 'plotly_dark',autosize=False, width=1500, height=400)
fig.update_layout(title=title, xaxis_title='date of start of strategy implementation', yaxis_title='Outperformance (%)')
fig.show()

# ANNEX: ADDITIONAL VERIFICATION

In [258]:
# Download the dataframe with the results
params.to_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/GebertIndicatorData.xlsx')